In [3]:
import csv
import numpy as np
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import multilabel_confusion_matrix
import boto3
import os
import pandas as pd
import shutil

In [26]:
def calc_conf_matrix(filename):
  view = filename[50:][:-18]
  print(view)
  pred = {}
  # read in txt file
  with open(filename, newline = '') as data:                                                                                          
      data_reader = csv.reader(data, delimiter='\t')
      firstline = True
      for line in data_reader:
          if firstline:    #skip first line
            firstline = False
            continue
          key = str(line[1][:-8])
          if key not in pred.keys():
              pred[key] = []
          pred[key].append(line[3]) #plax 1
          pred[key].append(line[7]) #psax 2
          pred[key].append(line[9]) #a2c_laocc 3
          pred[key].append(line[10]) #a2c 4
          pred[key].append(line[12]) #a3c_laocc 5
          pred[key].append(line[13]) #a3c 6
          pred[key].append(line[15]) #a4c_laocc 7
          pred[key].append(line[16]) #a4c 8
  
  # convert to labels
  thresh = 0.80
  for key in pred.keys():
      for i in range(0, 8):
        if float(pred[key][i]) > thresh:
            pred[key][i] = i+1; #plax
        else:
          pred[key][i] = 0;
  y_true = np.array([0, 0, 0, 0, 0, 0, 0, 0])

  if view == "plax":
    y_true = np.array([1, 0, 0, 0, 0, 0, 0, 0])
  if view == "psax":
    y_true = np.array([0, 2, 0, 0, 0, 0, 0, 0])
  if view == "a2c_laocc":
    y_true = np.array([0, 0, 3, 0, 0, 0, 0, 0])
  if view == "a2c":
    y_true = np.array([0, 0, 0, 4, 0, 0, 0, 0])
  if view == "a3c_laocc":
    y_true = np.array([0, 0, 0, 0, 5, 0, 0, 0])
  if view == "a3c":
    y_true = np.array([0, 0, 0, 0, 0, 6, 0, 0])
  if view == "a4c_laocc":
    y_true = np.array([0, 0, 0, 0, 0, 0, 7, 0])
  if view == "a4c":
    y_true = np.array([0, 0, 0, 0, 0, 0, 0, 8])

  c_all = [[0, 0, 0], [0, 0, 0]]
  for key in pred.keys():
      y_pred = np.array(pred[key])
      c = confusion_matrix(y_pred, y_true)
      if c.shape == (1,1):
            c = np.pad(c, (0,1), 'constant', constant_values=(0))
      for i in range(0,2):
          for j in range(0,2):
              c_all[i][j] = c_all[i][j] + c[i][j]; 
  
  return c_all     

In [24]:
def err(FP, FN, P, N):
  e = (FP + FN)/(P+N)
  return e

def acc(TP, TN, P, N):
  a = (TP + TN)/(P + N)
  return a

In [27]:
viewdirs = os.listdir("outputs/")
for view in viewdirs:
    print(view)
    filename = "outputs/" + view + "/results/" + view + "_probabilities.txt"
    cfm = calc_conf_matrix(filename)
    print(cfm)
    Fp = cfm[0][1]
    Fn = cfm[1][0]
    Tp = cfm[0][0]
    Tn = cfm[1][1]
    P = Tp + Fp
    N = Tn + Fn
    print("Error:", err(Fp, Fn, P, N))
    print("Accuracy:", acc(Tp, Tn, P, N))



a2c

[[684, 0, 0], [44, 0, 0]]
Error: 0.06043956043956044
Accuracy: 0.9395604395604396
a4c

[[1052, 0, 0], [148, 0, 0]]
Error: 0.12333333333333334
Accuracy: 0.8766666666666667
psax

[[941, 0, 0], [123, 0, 0]]
Error: 0.1156015037593985
Accuracy: 0.8843984962406015
a3c

[[945, 0, 0], [79, 0, 0]]
Error: 0.0771484375
Accuracy: 0.9228515625
a4c_laocc

[[1653, 0, 0], [235, 0, 0]]
Error: 0.12447033898305085
Accuracy: 0.8755296610169492
a3c_laocc

[[1318, 0, 0], [130, 0, 0]]
Error: 0.08977900552486189
Accuracy: 0.9102209944751382
plax

[[1889, 0, 0], [223, 0, 0]]
Error: 0.10558712121212122
Accuracy: 0.8944128787878788
a2c_laocc

[[1676, 0, 0], [172, 0, 0]]
Error: 0.09307359307359307
Accuracy: 0.9069264069264069
